# Case Teddy

<p> Este Notebook tem por objetivo:</p>

- Coletar os dados da API e carrega-los dentro de um Data Lake do banco de dados armazenado no Postgres;

- Somente as linhas de dados completas devem ser armazenadas no Data Lake, linhas incompletas devem ser excluídas;

- Criar string de conexão da engine;

- Criar o script em SQL para a tabela

<p> Para o fluxo a seguir subentede-se que o BD já possue toda estrutura proposta criada sendo ela:

- Schema: teddy_360
    - Tabela: teddy_360

! Os scripts para criação do Schema e tabelas encontram-se na pasta deste projeto como *create_database.sql*

---

## Preparando o ambiente

In [1]:
## Descomente as linhas [!pip install ...] caso precise instalar algumas das bibliotecas utilizadas

## Instalar pacotes do requests
# !pip install -q requests

## Instalar pacotes do requests
# !pip install -q pandas

## Instalar Watermark
# !pip install -q watermark

## Instalar o psycopg2-binary -- para suprir dependência da lib sqlalchemy
# !pip install psycopg2-binary -q

In [18]:
# Bibliotecas utilizadas

import sqlalchemy as db         # Acessar e gerenciar BD
import requests                 # Obter dados API - WEB
import pandas as pd             # Manipular dados
from datetime import datetime   # Manipular Data/hora

# *Caso não seja possível importar alguma das bibliotecas acima,
#  descomente a linha correspondente na célula acima

In [12]:
#versões dos pacotes usados neste notebook
%reload_ext watermark
%watermark --iversions

sqlalchemy: 2.0.30
pandas    : 2.2.2
requests  : 2.32.2



**VERSÃO DAS LIB'S UTILIZADAS**
- sqlalchemy: 2.0.30
- pandas    : 2.2.2
- requests  : 2.32.2

---

## Configurando as conexão

In [13]:
# Definindo variáveis de conexão

## Fonte de dados
api_data = 'https://jsonplaceholder.typicode.com/todos/'

## Conexão BD
db_url = db.URL.create(
    "postgresql",
    username="postgres",
    password="Password123",
    host="localhost",
    port=5432,
    database="datalake",
    )

# Criando conexão com BD Postgres
engine = db.create_engine(db_url)

## Baixando dados da API

In [19]:
# Obtendo os dados da API
response = requests.get(api_data)
data = response.json()
df = pd.DataFrame(data)

# *** EXTRA ***
# AQUI SERÁ INCLUÍDO A DATA/HORA A CADA LINHA DE DADO A FIM DE MELHORAR A QUALIDADE DOS DADOS
df.insert(4, column="datetime_update", value=datetime.now())

# Descrevendo dados baixados
colunas = list(df.columns)
print("O dataset baixado possue:\n Linhas: ", df.shape[0],
       "\n Colunas: ", df.shape[1], colunas)


O dataset baixado possue:
 Linhas:  200 
 Colunas:  5 ['userId', 'id', 'title', 'completed', 'datetime_update']


## Verificando dados baixados

In [15]:
# Amostra dos dados
df.sample()

,userId,id,title,completed
94,5,95,vel nihil et molestiae iusto assumenda nemo qu...,True


In [16]:
# Verifica dados nulos no dataset
df.isna().value_counts()

userId  id     title  completed
False   False  False  False        200
Name: count, dtype: int64

### EXTRA - Verificando valores existentes no BD

<p> Esta etapa tem por objetivo prevenir que a tabela duplique dados em caso de atualizações.

In [ ]:
# Obtendo valores atuais no BD
with engine.connect() as conn:
   df_local = conn.execute(db.text("SELECT * FROM teddy_360.teddy_360")).fetchall()


# Transformando valores em um dicionario para criação de Dataframe para comparação
data_dict = [{'userId': userId, 
            'id': id, 
            'title':title, 
            'completed':completed, 
            'datetime_update':datetime_update} 
            for userId, id, title, completed, datetime_update in df_local]

# Criando DF de referência com dados da BD
df_local = pd.DataFrame(data_dict)

# Concatenando valores dos dois DF
df_updated = pd.concat([df_local, df], ignore_index=True)

# Removendo duplicados considerando a coluna 'id' como chave primária para nesta tabela
# caso os valores de um 'id' seja alterado preserva-se o último registro
df_updated.drop_duplicates(subset=['id'], keep='last', inplace=True)

## Exportando dados para o Banco de dados

In [ ]:
# Remove valores nulos do dataset
df_updated.dropna(inplace=True)

# Envia dados para o BD 
## Foram selecionados apenas as colunas existentes na API até o momento para evitar falha na carga de dados
## Caso sejam incluídos novas colunas na API as mesmas devem ser incluídas no BD e na linha abaixo

colunas_api = ['userId', 'id', 'title', 'completed', "datetime_update"]     # Colunas mapeadas até o momento 

df_updated[colunas_api].to_sql('teddy_360', engine, schema='teddy_360', if_exists='replace', index=False)

# Verificando se todos os dados foram gravados no BD
with engine.connect() as conn:
   query = conn.execute(db.text("SELECT * FROM teddy_360.teddy_360")).fetchall()

if len(query) == len(df):
   print(f"Todas as {len(query)} foram importadas com sucesso!")
elif len(query) > len(df):
   print(f"Foi incluida {len(query) - len(df)} linha(s)!")
else:
   print(f"Faltaram {abs(len(df) - len(query))}")